<a href="https://colab.research.google.com/github/lakshh670/100_days_of_DeepLearning/blob/main/Integer_encoding_simple_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# In this notebook we will make a simple rnn model.

In [1]:
import numpy as np
# But first we see how to encode the text. Let's say we have a set of reviews and we need to sentimental analysis of that. 1st we need to convert the text into tokens.

docs = ['go india',
		'india india',
		'hip hip hurray',
		'jeetega bhai jeetega india jeetega',
		'bharat mata ki jai',
		'kohli kohli',
		'sachin sachin',
		'dhoni dhoni',
		'modi ji ki jai',
		'inquilab zindabad']

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

In [5]:
tokenizer=Tokenizer(oov_token='<nothing>')

In [13]:
tokenizer.fit_on_texts(docs)

In [14]:
tokenizer.word_index

{'<nothing>': 1,
 'india': 2,
 'jeetega': 3,
 'hip': 4,
 'ki': 5,
 'jai': 6,
 'kohli': 7,
 'sachin': 8,
 'dhoni': 9,
 'go': 10,
 'hurray': 11,
 'bhai': 12,
 'bharat': 13,
 'mata': 14,
 'modi': 15,
 'ji': 16,
 'inquilab': 17,
 'zindabad': 18}

In [15]:
tokenizer.word_counts

OrderedDict([('go', 1),
             ('india', 4),
             ('hip', 2),
             ('hurray', 1),
             ('jeetega', 3),
             ('bhai', 1),
             ('bharat', 1),
             ('mata', 1),
             ('ki', 2),
             ('jai', 2),
             ('kohli', 2),
             ('sachin', 2),
             ('dhoni', 2),
             ('modi', 1),
             ('ji', 1),
             ('inquilab', 1),
             ('zindabad', 1)])

In [18]:
sequences=tokenizer.texts_to_sequences(docs)
sequences
# We have 10 sentences(reviews) in the docs and each sentence has been tokenized. We can see that 1st sentence has two words so the length of the list representing that sentenece is also 2.

[[10, 2],
 [2, 2],
 [4, 4, 11],
 [3, 12, 3, 2, 3],
 [13, 14, 5, 6],
 [7, 7],
 [8, 8],
 [9, 9],
 [15, 16, 5, 6],
 [17, 18]]

In [19]:
# Now we need to make the length of each sentence(also called as timestamp) because the model has fixed amount of input nodes. We wil do zero padding.
from keras.utils import pad_sequences
sequences=pad_sequences(sequences,padding='post') # if padding='pre' then the zeros will come at the starting.
sequences

array([[10,  2,  0,  0,  0],
       [ 2,  2,  0,  0,  0],
       [ 4,  4, 11,  0,  0],
       [ 3, 12,  3,  2,  3],
       [13, 14,  5,  6,  0],
       [ 7,  7,  0,  0,  0],
       [ 8,  8,  0,  0,  0],
       [ 9,  9,  0,  0,  0],
       [15, 16,  5,  6,  0],
       [17, 18,  0,  0,  0]], dtype=int32)

In [20]:
# Now let's build a simple RNN model.
from keras.datasets import imdb
from keras import Sequential
from keras.layers import Dense, SimpleRNN

In [22]:
(x_train,y_train),(x_test,y_test)=imdb.load_data()
# This dataset is already tokenized

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [24]:
x_train.shape
# i.e. we have total 25000 reviews.

(25000,)

In [27]:
len(x_train[0])
# i.e. no. of words in first review is 218

218

In [29]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

In [32]:
x_train = pad_sequences(x_train,padding='post',maxlen=50)
x_test = pad_sequences(x_test,padding='post',maxlen=50)

In [33]:
#Model Building
model=Sequential()
model.add(SimpleRNN(32,input_shape=(50,1),return_sequences=False,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,121 (4.38 KB)

 Trainable params: 1,121 (4.38 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [35]:
model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.5008 - loss: 75.3507 - val_accuracy: 0.5072 - val_loss: 16.4181
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.5030 - loss: 17.4610 - val_accuracy: 0.4938 - val_loss: 14.3290
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.5025 - loss: 12.1398 - val_accuracy: 0.5038 - val_loss: 19.7001
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.5081 - loss: 11.4519 - val_accuracy: 0.4934 - val_loss: 9.9928
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.4969 - loss: 9.4197 - val_accuracy: 0.4987 - val_loss: 6.6899
